# prepare feature

In [3]:
import os

import numpy as np
import torch

from tqdm import tqdm

### collect .pdb files

In [5]:
import subprocess

data_pdb_path = "./data/DMS/1AO7/pdb/"
os.makedirs(data_pdb_path, exist_ok=True)

os.system(f"cp ./datasets/DMS/1AO7/cleaned_PDBs/*.pdb {data_pdb_path}")
os.system(f"cp ./datasets/DMS/1AO7/mut_PDBs/*.pdb {data_pdb_path}")

print(f"Total .pdb file: {subprocess.check_output(f'ls {data_pdb_path} | wc -w', shell=True)}")


Total .pdb file: b'13434\n'


### extract sequence & coordinate from .pdb file

In [6]:
import glob
from utils import *

from Bio.PDB import PDBParser

data_seq_path = "./data/DMS/1AO7/seq/"
data_coord_path = "./data/DMS/1AO7/coord/"
os.makedirs(data_seq_path, exist_ok=True)
os.makedirs(data_coord_path, exist_ok=True)

pdb_list = glob.glob(f"{data_pdb_path}/*.pdb")
for pdb_filepath in tqdm(pdb_list):
    name = os.path.basename(pdb_filepath).split(".")[0]
    # print(name)
    
    parser = PDBParser(QUIET=True)
    struct = parser.get_structure(name, pdb_filepath)
    res_list = get_clean_res_list(struct.get_residues(), verbose=False, ensure_ca_exist=True)
    # ensure all res contains N, CA, C and O
    res_list = [res for res in res_list if (('N' in res) and ('CA' in res) and ('C' in res) and ('O' in res))]

    # extract sequence
    seq = "".join([three_to_one.get(res.resname) for res in res_list])

    # extract coordinate
    coord = []
    for res in res_list:
        res_coord = []
        R_group = []
        for atom in res:
            if atom.get_name() in ['N', 'CA', 'C', 'O']:
                res_coord.append(atom.get_coord())
            else:
                R_group.append(atom.get_coord())

        if len(R_group) == 0:
            R_group.append(res['CA'].get_coord())
        R_group = np.array(R_group).mean(axis=0)
        res_coord.append(R_group)
        coord.append(res_coord)
    coord = np.array(coord)  # convert list directly to tensor would be rather slow, suggest use ndarray as transition
    coord = torch.tensor(coord, dtype=torch.float32)

    # save to file
    seq_to_file = f"{data_seq_path}/{name}.txt"
    coord_to_file = f"{data_coord_path}/{name}.pt"
    with open(seq_to_file, "w") as seq_file:
        seq_file.write(seq)
    torch.save(coord, coord_to_file)


### extract ProtTrans feature from sequence

In [1]:
import gc
from tqdm import tqdm
import torch
from transformers import T5Tokenizer, T5EncoderModel

In [2]:
ProtTrans_toolpath = "./tools/Prot-T5-XL-U50/"
gpu = '1'

# Load the vocabulary and ProtT5-XL-UniRef50 Model
tokenizer = T5Tokenizer.from_pretrained(ProtTrans_toolpath, do_lower_case=False)
model = T5EncoderModel.from_pretrained(ProtTrans_toolpath)
gc.collect()

# Load the model into the GPU if avilabile and switch to inference mode
device = torch.device('cuda:' + gpu if torch.cuda.is_available() and gpu else 'cpu')
model = model.to(device)
model = model.eval()


Some weights of the model checkpoint at ./tools/Prot-T5-XL-U50/ were not used when initializing T5EncoderModel: ['decoder.block.7.layer.0.SelfAttention.o.weight', 'decoder.block.8.layer.1.EncDecAttention.v.weight', 'decoder.block.20.layer.2.DenseReluDense.wi.weight', 'decoder.block.16.layer.1.EncDecAttention.k.weight', 'decoder.block.11.layer.0.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.o.weight', 'decoder.block.17.layer.1.EncDecAttention.o.weight', 'decoder.block.6.layer.1.EncDecAttention.v.weight', 'decoder.block.17.layer.2.layer_norm.weight', 'decoder.block.8.layer.0.SelfAttention.o.weight', 'decoder.block.22.layer.2.DenseReluDense.wi.weight', 'decoder.block.6.layer.2.DenseReluDense.wi.weight', 'decoder.block.22.layer.2.DenseReluDense.wo.weight', 'decoder.block.8.layer.2.DenseReluDense.wo.weight', 'decoder.block.7.layer.0.layer_norm.weight', 'decoder.block.8.layer.1.EncDecAttention.q.weight', 'decoder.block.15.layer.1.EncDecAttention.v.weight', 'decoder.block.8.layer

In [9]:
data_seq_path = "./data/DMS/1AO7/seq/"

data_ProtTrans_raw_path = "./data/DMS/1AO7/ProtTrans_raw/"
os.makedirs(data_ProtTrans_raw_path, exist_ok=True)

pdb_list = glob.glob(f"{data_pdb_path}/*.pdb")
name_list = [os.path.basename(pdb_filepath).split(".")[0] for pdb_filepath in pdb_list]

for name in tqdm(name_list):
    # ProtTrans_to_file = f"{data_ProtTrans_raw_path}/{name}.npy"
    # if os.path.exists(ProtTrans_to_file):
    #     continue
    with open(f"{data_seq_path}/{name}.txt") as seq_file:
        seq = seq_file.readline()
    batch_name_list = [name]
    batch_seq_list = [" ".join(list(seq))]
    # print(len(seq))
    # print(batch_name_list)
    # print(batch_seq_list)

    # Tokenize, encode sequences and load it into the GPU if possibile
    ids = tokenizer.batch_encode_plus(batch_seq_list, add_special_tokens=True, padding=True)
    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)

    # Extracting sequences' features and load it into the CPU if needed
    with torch.no_grad():
        embedding = model(input_ids=input_ids,attention_mask=attention_mask)
    embedding = embedding.last_hidden_state.cpu()

    # Remove padding (\<pad>) and special tokens (\</s>) that is added by ProtT5-XL-UniRef50 model
    for seq_num in range(len(embedding)):
        seq_len = (attention_mask[seq_num] == 1).sum()
        seq_emb = embedding[seq_num][:seq_len-1]
        # print(f"truncate padding: {embedding[seq_num].shape} -> {seq_emb.shape}")
        ProtTrans_to_file = f"{data_ProtTrans_raw_path}/{batch_name_list[seq_num]}.npy"
        np.save(ProtTrans_to_file, seq_emb)


100%|██████████| 13434/13434 [34:35<00:00,  6.47it/s] 


#### normalize raw ProtTrans

In [10]:
Max_protrans = []
Min_protrans = []
for name in tqdm(name_list):
    raw_protrans = np.load(f"{data_ProtTrans_raw_path}/{name}.npy")
    Max_protrans.append(np.max(raw_protrans, axis = 0))
    Min_protrans.append(np.min(raw_protrans, axis = 0))

Min_protrans = np.min(np.array(Min_protrans), axis = 0)
Max_protrans = np.max(np.array(Max_protrans), axis = 0)
print(Min_protrans)
print(Max_protrans)

np.save("./data/DMS/1AO7/Max_ProtTrans_repr.npy", Max_protrans)
np.save("./data/DMS/1AO7/Min_ProtTrans_repr.npy", Min_protrans)

100%|██████████| 13434/13434 [07:39<00:00, 29.24it/s] 


[-0.9715796 -0.53459   -0.7825786 ... -0.7357477 -0.7062289 -0.7532639]
[0.6705426  0.6439474  0.8558858  ... 0.924354   0.8515873  0.69065195]


In [11]:
Max_protrans = np.load("./data/DMS/1AO7/Max_ProtTrans_repr.npy")
Min_protrans = np.load("./data/DMS/1AO7/Min_ProtTrans_repr.npy")

data_ProtTrans_path = "./data/DMS/1AO7/ProtTrans/"
os.makedirs(data_ProtTrans_path, exist_ok=True)

for name in tqdm(name_list):
    raw_protrans = np.load(f"{data_ProtTrans_raw_path}/{name}.npy")
    protrans = (raw_protrans - Min_protrans) / (Max_protrans - Min_protrans)
    ProtTrans_to_file = f"{data_ProtTrans_path}/{name}.pt"
    torch.save(torch.tensor(protrans, dtype = torch.float32), ProtTrans_to_file)


  0%|          | 0/13434 [00:00<?, ?it/s]

100%|██████████| 13434/13434 [10:53<00:00, 20.57it/s] 


### extract DSSP feature from .pdb file

#### correct format of mut.pdb: col of Occupancy(55 - 60) should be "{:.2f}"

In [12]:
data_pdb_path = "./data/DMS/1AO7/pdb/"

for name in tqdm(name_list):
    pdb_filepath = f"{data_pdb_path}/{name}.pdb"

    with open(pdb_filepath, "r") as f:
        lines = f.readlines()

    for i in range(len(lines)):
        if lines[i].split()[0] == "REMARK":
            continue
        lines[i] = lines[i][:57] + '.00' + lines[i][60:]

    with open(pdb_filepath, "w") as f:
        f.writelines(lines)


100%|██████████| 13434/13434 [08:42<00:00, 25.73it/s]


In [14]:
from utils import *

data_pdb_path = "./data/DMS/1AO7/pdb/"
data_seq_path = "./data/DMS/1AO7/seq/"
dssp_toolpath = "./tools/mkdssp"

data_DSSP_path = "./data/DMS/1AO7/DSSP"
os.makedirs(data_DSSP_path, exist_ok=True)

for name in tqdm(name_list):
    pdb_filepath = f"{data_pdb_path}/{name}.pdb"
    with open(f"{data_seq_path}/{name}.txt") as seq_file:
        seq = seq_file.readline()

    DSSP_to_file = f"{data_DSSP_path}/{name}.dssp"
    dssp_cmd = f"{dssp_toolpath} -i {pdb_filepath} -o {DSSP_to_file}"
    os.system(dssp_cmd)

    try:
        dssp_seq, dssp_matrix = process_dssp(DSSP_to_file)
        # dssp_seq: likely equal to original sequence
        # dssp_matrix (list<ndarray>): list of (1, 9) vector, length of dssp_seq
        if dssp_seq != seq:
            dssp_matrix = match_dssp(dssp_seq, dssp_matrix, seq)
        
        DSSP_to_file = f"{data_DSSP_path}/{name}.pt"
        torch.save(torch.tensor(np.array(dssp_matrix), dtype = torch.float32), DSSP_to_file)
        # shape(AA_len, 9)
        # os.system("rm {DSSP_to_file}"")
    except:
        print(f"Wrong entry prompt: $ {dssp_cmd}")
        continue


100%|██████████| 13434/13434 [21:48<00:00, 10.27it/s]


# prepare dataset

In [5]:
import os
import glob
import pandas as pd
import torch

df = pd.DataFrame()
df["mut_name"] = glob.glob("./data/DMS/1AO7/pdb/1AO7_*.pdb")
df["mut_name"] = df["mut_name"].apply(lambda x: os.path.basename(x).split(".")[0])
df["wt_name"] = "1AO7"
df

,mut_name,wt_name
0,1AO7_GE61I,1AO7
1,1AO7_SD25Q,1AO7
2,1AO7_NE26A,1AO7
3,1AO7_LB40K,1AO7
4,1AO7_SE82R,1AO7
...,...,...
13428,1AO7_FA208I,1AO7
13429,1AO7_LA126S,1AO7
13430,1AO7_PA235M,1AO7
13431,1AO7_RA234P,1AO7


In [6]:
torch.save(df, "./data/DMS/1AO7/mut_wt_pairs.pt")